In [1]:
import pypdf
import pypdf.annotations

import math

PaperSizes are in pts, which are .35 mm

In [2]:
infilename = "/home/tbogue/Documents/Spells.pdf"

In [3]:
def make_sheet(writer, page1, page2, page3, page4):
    width = page1.mediabox[2]
    height = page1.mediabox[3]
    sheet1 = writer.add_blank_page(width = width * 2, height = height)
    #page1.mediabox = page2.mediabox = page3.mediabox = page4.mediabox = sheet1.mediabox
    sheet1.merge_transformed_page(
        page1,
        pypdf.Transformation().translate(width,0)
    )
    sheet1.merge_transformed_page(
        page4,
        pypdf.Transformation().translate(0, 0)
    )
    sheet2 = writer.add_blank_page(width = width * 2, height = height)
    sheet2.merge_transformed_page(
        page2,
        pypdf.Transformation().translate(0, 0)
    )
    sheet2.merge_transformed_page(
        page3,
        pypdf.Transformation().translate(width, 0)
    )
    

In [4]:
def signature_plan(num_pages):
    sheets_per_signature = 8
    # reserve 4 pages for binding
    total_pages = num_pages + 4
    # add extra pages to make an integral number of sheets
    num_sheets = math.ceil(total_pages / 4)
    total_pages = 4 * num_sheets
    num_signatures = math.ceil(num_sheets / sheets_per_signature)
    extra_pages = total_pages - num_pages
    pairs_of_extra_pages = math.ceil(extra_pages / 2)
    extra_front_pages = 2 * math.ceil(pairs_of_extra_pages / 2)
    extra_back_pages = extra_pages - extra_front_pages
    all_pages = [0] * extra_front_pages + list(range(1, num_pages + 1)) + [0] * extra_back_pages
    current_page = 0
    signatures = []
    for signatureNum in range(0, num_signatures):
        signature = []
        starting_page = current_page
        pages_left = total_pages - current_page
        sheets_left = int(pages_left / 4)
        signatures_left = num_signatures - signatureNum
        num_sheets_in_signature = math.ceil(sheets_left / signatures_left)
        ending_page = current_page + num_sheets_in_signature * 4 - 1
        for sheet_num in range(0, num_sheets_in_signature):
            sheet = [ 
                starting_page + sheet_num * 2,
                starting_page + sheet_num * 2 + 1,
                ending_page - sheet_num * 2 - 1,
                ending_page - sheet_num * 2
            ]
            signature.append(sheet)
        signatures.append(signature)
        current_page = ending_page + 1
    return({"page_list":all_pages, "signatures":signatures})

In [5]:
# returns a blank page if page_id is 0 or the page otherwise
def get_page(reader, page_id, reverse=False):
    if page_id == 0:
        width = reader.pages[0].mediabox[2]
        height = reader.pages[1].mediabox[3]
        return pypdf._page.PageObject.create_blank_page(height=height, width=width)
    else:
        page_editor = pypdf.PdfWriter()
        page = reader.pages[page_id - 1]
        page_editor.add_page(page)
        width = page.mediabox[2]
        rect = (5,5,23,20)
        if (page_id % 2 == 0) != reverse :
            rect = (width-rect[2], 5, width-rect[0], 20)
        page_num_annotation = pypdf.annotations.FreeText(
            rect=rect,
            text=f'{page_id}'
        )
        page_editor.add_annotation(page_number=0, annotation=page_num_annotation)
        page_editor.write("tmp.pdf")
        return pypdf.PdfReader("tmp.pdf").pages[0]

In [6]:
def convert_pdf(infilename, outfileName=None):
    if outfileName == None:
        outfileName = infilename[0:-4] + "-book.pdf"
    reader = pypdf.PdfReader(infilename)
    writer = pypdf.PdfWriter()
    plan = signature_plan(len(reader.pages))
    signatures = plan['signatures']
    page_list = plan['page_list']
    for signature in signatures:
        for sheet in signature:            
            page_nums = [
                page_list[sheet[0]],
                page_list[sheet[1]],
                page_list[sheet[2]],
                page_list[sheet[3]]
            ]
            make_sheet(
                writer, 
                get_page(reader, page_list[sheet[0]], True),
                get_page(reader, page_list[sheet[1]], True),
                get_page(reader, page_list[sheet[2]], True),
                get_page(reader, page_list[sheet[3]], True)
            )
    writer.write(outfileName)

In [7]:
convert_pdf(infilename)